<a href="https://colab.research.google.com/github/Guylord/APHRC_DSE_Inspire_Hackathon_2024/blob/main/APHRC_DSE_Inspire_Hackathon_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install recordlinkage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import recordlinkage as rl

In [4]:
hdss_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_hdss_v3.csv')
facility_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_facility_v3.csv')

print('HDSS INFO')
hdss_df.info()
print('\n\nFACILITY INFO')
facility_df.info()

HDSS INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   recnr       4115 non-null   int64  
 1   firstname   4115 non-null   object 
 2   lastname    4031 non-null   object 
 3   petname     2894 non-null   object 
 4   dob         4115 non-null   object 
 5   sex         4115 non-null   int64  
 6   nationalid  0 non-null      float64
 7   hdssid      4115 non-null   object 
 8   hdsshhid    4115 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 289.5+ KB


FACILITY INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recnr       2902 non-null   int64 
 1   firstname   2902 non-null   object
 2   lastname    2835 non-null   object
 3   petname     2511 non-null   object
 4   do

# *FACILITY dataset*

In [5]:
facility_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020


In [6]:
facility_df.set_index('recnr', inplace=True)

In [8]:
from recordlinkage.preprocessing import clean

for col in facility_df.select_dtypes('object').columns:
  clean(facility_df[col], replace_by_none='[^ \\-\\_A-Za-z0-9]+', remove_brackets=True)

In [9]:
facility_df['fullname'] = facility_df['firstname'].fillna('') + ' ' + facility_df['lastname'].fillna('') + ' ' + facility_df['petname'].fillna('')

In [10]:
facility_df['dob'] = pd.to_datetime(facility_df['dob'])
facility_df['visitdate'] = pd.to_datetime(facility_df['visitdate'])

<ipython-input-10-9b6fa05c9244>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  facility_df['visitdate'] = pd.to_datetime(facility_df['visitdate'])


In [11]:
facility_df.isna().sum()

firstname       0
lastname       67
petname       391
dob             0
sex             0
nationalid      0
patientid       0
visitdate       0
fullname        0
dtype: int64

In [12]:
facility_df.duplicated().sum()

0

In [13]:
facility_df[facility_df.duplicated(subset=['fullname', 'dob', 'sex'], keep=False)].sort_values('fullname').head(10)

,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate,fullname
recnr,,,,,,,,,
2686,Abubakali,Lunkumu,Jjunju,1934-07-08,1,N_ID_13055,4754,2019-01-06,Abubakali Lunkumu Jjunju
2583,Abubakali,Lunkumu,Jjunju,1934-07-08,1,N_ID_12749,4651,2022-12-21,Abubakali Lunkumu Jjunju
1162,Abuneli,Lawrence,Asega,1959-11-08,1,N_ID_8444,3231,2019-09-20,Abuneli Lawrence Asega
575,Abuneli,Lawrence,Asega,1959-11-08,1,N_ID_6689,2645,2022-03-07,Abuneli Lawrence Asega
2533,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12599,4601,2020-06-23,Aidah Fatuma Aya
2590,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12770,4658,2020-04-19,Aidah Fatuma Aya
629,Akello,Jesca,Barungi,1937-12-20,2,N_ID_6851,2699,2023-08-06,Akello Jesca Barungi
1299,Akello,Jesca,Barungi,1937-12-20,2,N_ID_8855,3368,2022-08-13,Akello Jesca Barungi
1633,Alex,Idinda,Nakato,1967-08-10,1,N_ID_9860,3702,2020-10-01,Alex Idinda Nakato


In [14]:
facility_df.duplicated(subset='nationalid', keep=False).sum()

0



* It's evident that some records have the same entries for 'firstname,' 'lastname,' 'petname,' and 'dob,' implying that they refer to the same entity but with different 'nationalid' entries during various visits to the facility. This suggests that individuals visiting the facility did not retain a single 'nationalid,' rendering 'nationalid' unusable as a unique identifier

* Individuals have different 'patientid' entries at each visit to the health facility






# *HDSS dataset*

In [15]:
hdss_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005


In [16]:
hdss_df.set_index('recnr', inplace=True)

In [17]:
for col in hdss_df.select_dtypes('object').columns:
  clean(hdss_df[col], replace_by_none='[^ \\-\\_A-Za-z0-9]+', remove_brackets=True)

In [18]:
hdss_df['fullname'] = hdss_df['firstname'].fillna('') + ' ' + hdss_df['lastname'].fillna('') + ' ' + hdss_df['petname'].fillna('')

In [19]:
hdss_df['dob'] = pd.to_datetime(hdss_df['dob'])

In [20]:
hdss_df.isna().sum()

firstname        0
lastname        84
petname       1221
dob              0
sex              0
nationalid    4115
hdssid           0
hdsshhid         0
fullname         0
dtype: int64

In [21]:
hdss_df.duplicated().sum()

0

In [22]:
mask = hdss_df.duplicated(subset=['fullname', 'dob', 'sex'], keep=False)
hdss_df[mask].sort_values('fullname')

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname
recnr,,,,,,,,,


In [23]:
hdss_df.duplicated(subset='hdssid').sum()

0

In [24]:
hdss_df.duplicated(subset='hdsshhid').sum()

0

In [ ]:
#indexer = rl.Index()
#indexer.block(left_on='dob', right_on='dob')
#pairs = indexer.index(facility_df, hdss_df)

In [ ]:
#indexer = rl.Index()
#indexer.block(left_on='fullname', right_on='fullname')
#pairs = indexer.index(facility_df, hdss_df)

In [43]:
indexer = rl.Index()
indexer.block(left_on='firstname', right_on='firstname')
pairs = indexer.index(hdss_df, facility_df)

In [ ]:
#indexer = rl.Index()
#indexer.block(left_on='sex', right_on='sex')
#pairs = indexer.index(facility_df, hdss_df)

In [45]:
compare = rl.Compare()

compare.string('firstname', 'firstname', method='jarowinkler', label='firstname')
compare.string('fullname', 'fullname', method='lcs', threshold = 0.70, label='fullname')
compare.date('dob', 'dob', label='dob')
compare.exact('sex', 'sex', label='sex')


potential_matches=compare.compute(pairs, hdss_df, facility_df)
potential_matches

firstname  fullname  dob  sex
recnr_1 recnr_2                               
1       429            1.0       0.0  0.0    1
        488            1.0       0.0  0.0    1
        552            1.0       0.0  0.0    1
        639            1.0       0.0  0.0    1
        1138           1.0       0.0  0.0    1
...                    ...       ...  ...  ...
4085    1550           1.0       1.0  0.0    1
4098    1033           1.0       1.0  1.0    1
        2869           1.0       1.0  0.0    1
4111    128            1.0       1.0  1.0    1
        1351           1.0       1.0  0.0    1

[34093 rows x 4 columns]

In [46]:
potential_matches.reset_index().duplicated(subset=['recnr_1']).sum()

30406

In [47]:
potential_matches.reset_index().duplicated(subset=['recnr_2']).sum()

31300

In [48]:
#lr=recordlinkage.LogisticRegressionClassifier()
#nb=recordlinkage.NaiveBayesClassifier()
#ecm=recordlinkage.ECMClassifier()
kmeans=rl.KMeansClassifier()
kmeans.fit(potential_matches)
matched_indices = kmeans.predict(potential_matches)
len(matched_indices)

3531

In [54]:
hdss_facility = pd.concat([hdss_df.loc[matched_indices.get_level_values(0)], facility_df.loc[matched_indices.get_level_values(1)]])
hdss_facility.sort_values('firstname').head(30)

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
3482,Ababu,Crispus,Lwasa,2006-04-23,1,NaN,I23482,HH103482,Ababu Crispus Lwasa,NaN,NaT
1209,Ababu,Crispus,Lwasa,2006-04-23,1,N_ID_8585,NaN,NaN,Ababu Crispus Lwasa,3278.0,2020-01-15
1217,Abalyogera,Namulondo,NaN,1941-06-07,2,NaN,I21217,HH101217,Abalyogera Namulondo,NaN,NaT
594,Abalyogera,Namulondo,Omutima,1941-07-13,2,N_ID_6746,NaN,NaN,Abalyogera Namulondo Omutima,2664.0,2019-01-18
655,Abasa,Bukosi,Jajja,1974-05-07,1,NaN,I20655,HH100655,Abasa Bukosi Jajja,NaN,NaT
989,Abasa,Bukosi,Awiti,1974-12-07,1,N_ID_7928,NaN,NaN,Abasa Bukosi Awiti,3058.0,2019-07-14
680,Abatuka,Nahiya,Fanda,1985-02-25,2,N_ID_7004,NaN,NaN,Abatuka Nahiya Fanda,2750.0,2018-08-29
3246,Abatuka,Nahiya,Fanda,1985-02-25,2,NaN,I23246,HH103246,Abatuka Nahiya Fanda,NaN,NaT
3282,Abbo,Shatra,NaN,1934-06-05,2,NaN,I23282,HH103282,Abbo Shatra,NaN,NaT
1393,Abbo,Shatra,Kisanja,1934-04-16,2,N_ID_9137,NaN,NaN,Abbo Shatra Kisanja,3462.0,2022-02-24


In [39]:
hdss_facility.shape

(7062, 11)

In [51]:
hdss_facility.duplicated().sum()

2311

In [52]:
hdss_facility.drop_duplicates(inplace=True)

In [53]:
hdss_facility.shape

(4751, 11)